Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [0]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [0]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using [nn.l2_loss(t)](https://www.tensorflow.org/api_docs/python/tf/nn/l2_loss). The right amount of regularization should improve your validation / test accuracy.

---

In [0]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  tf_n_layers = tf.constant(2)
  tf_regularize_rate = tf.constant(0.001)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([tf_n_layers, image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([tf_n_layers, num_labels]))
  
  def compute(dataset, weights, biases):
    logits = tf.nn.relu(tf.matmul(dataset, weights[0]) + biases[0])
    logits = tf.matmul(dataset, weights[1]) + biases[1]
    return logits

  # Training computation.
  logits = compute(tf_train_dataset, weights, biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + tf_regularize_rate * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(compute(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(compute(tf_test_dataset, weights, biases))

In [34]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")

  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 27.805964
Minibatch accuracy: 10.2%
Validation accuracy: 11.2%
Minibatch loss at step 500: 5.254585
Minibatch accuracy: 64.8%
Validation accuracy: 76.6%
Minibatch loss at step 1000: 2.768672
Minibatch accuracy: 75.8%
Validation accuracy: 78.5%
Minibatch loss at step 1500: 2.035361
Minibatch accuracy: 78.9%
Validation accuracy: 79.0%
Minibatch loss at step 2000: 1.301173
Minibatch accuracy: 80.5%
Validation accuracy: 81.0%
Minibatch loss at step 2500: 1.012961
Minibatch accuracy: 85.9%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 0.978090
Minibatch accuracy: 81.2%
Validation accuracy: 82.2%
Test accuracy: 89.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [0]:
batch_size = 10

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  tf_n_layers = tf.constant(2)
  tf_regularize_rate = tf.constant(0.001)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([tf_n_layers, image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([tf_n_layers, num_labels]))
  
  def compute(dataset, weights, biases):
    logits = tf.nn.relu(tf.matmul(dataset, weights[0]) + biases[0])
    logits = tf.matmul(dataset, weights[1]) + biases[1]
    return logits

  # Training computation.
  logits = compute(tf_train_dataset, weights, biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + tf_regularize_rate * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(compute(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(compute(tf_test_dataset, weights, biases))

In [37]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")

  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 40.255978
Minibatch accuracy: 0.0%
Validation accuracy: 8.2%
Minibatch loss at step 500: 6.455092
Minibatch accuracy: 70.0%
Validation accuracy: 73.1%
Minibatch loss at step 1000: 5.401825
Minibatch accuracy: 80.0%
Validation accuracy: 74.6%
Minibatch loss at step 1500: 3.898789
Minibatch accuracy: 60.0%
Validation accuracy: 69.2%
Minibatch loss at step 2000: 0.902420
Minibatch accuracy: 100.0%
Validation accuracy: 72.5%
Minibatch loss at step 2500: 0.952710
Minibatch accuracy: 80.0%
Validation accuracy: 71.1%
Minibatch loss at step 3000: 1.818704
Minibatch accuracy: 70.0%
Validation accuracy: 74.8%
Test accuracy: 81.8%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides [nn.dropout()](https://www.tensorflow.org/api_docs/python/tf/nn/dropout) for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [0]:
batch_size = 10

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  tf_n_layers = tf.constant(2)
  tf_dropout_rate = tf.constant(0.001)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([tf_n_layers, image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([tf_n_layers, num_labels]))
  
  def compute(dataset, weights, biases, dropout=False):
    weight0 = weights[0]
    weight1 = weights[1]
    biases0 = biases[0]
    biases1 = biases[1]
    if dropout == True:
      weight0 = tf.nn.dropout(weights[0], 0.5)
      weight1 = tf.nn.dropout(weights[1], 0.5)
      biases0 = tf.nn.dropout(biases[0], 0.5)
      biases1 = tf.nn.dropout(biases[1], 0.5)
    logits = tf.nn.relu(tf.matmul(dataset, weight0) + biases0)
    logits = tf.matmul(dataset, weight1) + biases1
    return logits

  # Training computation.
  logits = compute(tf_train_dataset, weights, biases, dropout=True)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(compute(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(compute(tf_test_dataset, weights, biases))

In [46]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")

  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 35.226173
Minibatch accuracy: 0.0%
Validation accuracy: 12.6%
Minibatch loss at step 500: 9.841374
Minibatch accuracy: 50.0%
Validation accuracy: 77.3%
Minibatch loss at step 1000: 10.906124
Minibatch accuracy: 50.0%
Validation accuracy: 78.9%
Minibatch loss at step 1500: 6.152736
Minibatch accuracy: 70.0%
Validation accuracy: 78.6%
Minibatch loss at step 2000: 2.288960
Minibatch accuracy: 80.0%
Validation accuracy: 77.1%
Minibatch loss at step 2500: 0.373692
Minibatch accuracy: 90.0%
Validation accuracy: 78.5%
Minibatch loss at step 3000: 3.006389
Minibatch accuracy: 80.0%
Validation accuracy: 77.2%
Test accuracy: 84.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [0]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  tf_n_layers = tf.constant(2)
  tf_regularize_rate = tf.constant(0.001)
  
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(
      0.5, # learning_rate
      global_step, # global_step
      1000, # decay_steps
      0.9, # decay_rate
      staircase=True
  )
    
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([tf_n_layers, image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([tf_n_layers, num_labels]))
  
  def compute(dataset, weights, biases):
    logits = tf.nn.relu(tf.matmul(dataset, weights[0]) + biases[0])
    logits = tf.matmul(dataset, weights[1]) + biases[1]
    return logits

  # Training computation.
  logits = compute(tf_train_dataset, weights, biases)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
    + tf_regularize_rate * tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(compute(tf_valid_dataset, weights, biases))
  test_prediction = tf.nn.softmax(compute(tf_test_dataset, weights, biases))

In [51]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")

  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)

    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))

  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 26.170269
Minibatch accuracy: 4.7%
Validation accuracy: 11.3%
Minibatch loss at step 500: 5.011247
Minibatch accuracy: 68.8%
Validation accuracy: 75.9%
Minibatch loss at step 1000: 2.901182
Minibatch accuracy: 73.4%
Validation accuracy: 78.2%
Minibatch loss at step 1500: 2.181290
Minibatch accuracy: 75.8%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 1.349906
Minibatch accuracy: 84.4%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 1.125248
Minibatch accuracy: 80.5%
Validation accuracy: 81.3%
Minibatch loss at step 3000: 1.052812
Minibatch accuracy: 80.5%
Validation accuracy: 82.1%
Test accuracy: 89.1%
